# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [49]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [50]:
locations_df = pd.read_csv('../output_data/cities.csv')
locations_df


,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,uthal,25.81,66.62,84.04,14,5,1.86,PK,1602790632
1,jamestown,42.10,-79.24,55.99,76,90,13.87,US,1602790447
2,hermanus,-34.42,19.23,55.99,88,100,1.01,ZA,1602790632
3,sangar,63.92,127.47,19.20,96,100,7.20,RU,1602790632
4,avera,33.19,-82.53,82.40,69,40,4.70,US,1602790633
...,...,...,...,...,...,...,...,...,...
74,bambous virieux,-20.34,57.76,75.00,83,40,8.05,MU,1602790655
75,tiksi,71.69,128.87,20.17,98,95,0.89,RU,1602790655
76,provideniya,64.38,-173.30,44.60,93,90,17.90,RU,1602790655
77,karratha,-20.74,116.85,74.03,38,0,8.55,AU,1602790415


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [51]:
locations = locations_df[["Lat", "Lon"]]

heat = locations_df["Humidity"]

In [52]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights = heat, dissipating = False, max_intensity = 10, point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [53]:
ideal_index = locations_df[(locations_df["Max Temp"] < 80) & (locations_df["Max Temp"] > 70) & (locations_df["Wind Speed"] < 10) & (locations_df["Cloudiness"] < 40)].index 

ideal_index

Int64Index([15, 27, 31, 66, 77, 78], dtype='int64')

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [54]:
hotel_df = locations_df.iloc[ideal_index, :]

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

hotel_df

,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
15,pasighat,28.07,95.33,75.49,70,3,3.06,IN,1602790636
27,ribeira grande,38.52,-28.70,71.60,73,20,4.70,PT,1602790640
31,itoman,26.12,127.67,77.00,78,20,8.05,JP,1602790641
66,sambava,-14.27,50.17,73.81,80,1,9.78,MG,1602790652
77,karratha,-20.74,116.85,74.03,38,0,8.55,AU,1602790415
78,padampur,29.68,73.62,74.39,20,0,2.37,IN,1602790656


In [55]:
for index, row in hotel_df.iterrows():
    
    lat = row["Lat"]
    lon = row["Lon"]
    
    params = {
    "location": f"{lat}, {lon}",
    "name": "hotel",
    "radius": 5000,
    "key": g_key
}
    response = requests.get(base_url, params = params)
    #print(index)
    
    hotel_data = response.json()
    print(f"Reading row: {index}")
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_data["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/results....skipping.")

Reading row: 15
Reading row: 27
Reading row: 31
Reading row: 66
Reading row: 77
Reading row: 78
Missing field/results....skipping.


In [56]:
hotel_df

,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
15,pasighat,28.07,95.33,75.49,70,3,3.06,IN,1602790636,"The Serene Abode, Pasighat (A.P)"
27,ribeira grande,38.52,-28.70,71.60,73,20,4.70,PT,1602790640,Quinta Das Buganvílias
31,itoman,26.12,127.67,77.00,78,20,8.05,JP,1602790641,Southern Beach Hotel & Resort Okinawa
66,sambava,-14.27,50.17,73.81,80,1,9.78,MG,1602790652,Melrose
77,karratha,-20.74,116.85,74.03,38,0,8.55,AU,1602790415,Karratha International Hotel
78,padampur,29.68,73.62,74.39,20,0,2.37,IN,1602790656,NaN


In [57]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lon"]]

In [58]:
# Add marker layer ontop of heat map
marker_locations = hotel_df[["Lat", "Lon"]]
markers = gmaps.marker_layer(marker_locations, hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))